<a href="https://colab.research.google.com/github/Corassini/Curso_deep_learning/blob/main/Exercicio_selecao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Abrindo o Arquivo CISI

In [1]:
!tar xf cisi.tar.gz

In [2]:
!pwd

/content


In [3]:
!ls

CISI.ALL  CISI.BLN  CISI.QRY  CISI.REL	cisi.tar.gz  sample_data


### Lendo os arquivos CISI e separando em diferentes variáveis

In [4]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer

import nltk
nltk.download('stopwords')
nltk.download('punkt')


stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer() 
tokenizer = RegexpTokenizer(r'\w+')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
with open('/content/CISI.ALL','r') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")

doc_set = {}
doc_id = ""
doc_text = ""

for l in lines:
    if l.startswith(".I"):
        doc_id = l.split(" ")[1].strip() 
    elif l.startswith(".X"):
        doc_set[doc_id] = doc_text.lstrip(" ")
        doc_id = ""
        doc_text = ""
    else:
        doc_text += l.strip()[3:] + " " 

documents = []
for key, value in doc_set.items():
  documents.append(value)
doc_token = []
for item in documents:
  doc_token.append([stemmer.stem(token) for token in tokenizer.tokenize(item.lower()) if token not in stop_words])

In [6]:
with open('/content/CISI.QRY', 'r') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")
      
qry_set = {}
qry_id = ""
for l in lines:
    if l.startswith(".I"):
        qry_id = l.split(" ")[1].strip() 
    elif l.startswith(".W"):
        qry_set[qry_id] = l.strip()[3:]
        qry_id = ""


queries = []
for key, value in qry_set.items():
  queries.append(value)
queries_token = []
for item in queries:
  queries_token.append([stemmer.stem(token) for token in tokenizer.tokenize(item.lower()) if token not in stop_words])

In [7]:
rel_set = {}
with open('/content/CISI.REL', 'r') as f:
    for l in f.readlines():
        qry_id = l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[0] 
        doc_id = l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[-1]

        if qry_id in rel_set:
            rel_set[qry_id].append(doc_id)
        else:
            rel_set[qry_id] = []
            rel_set[qry_id].append(doc_id)

print(f"\n\nNumber of mappings = {len(rel_set)}")
print(rel_set["1"]) 



Number of mappings = 76
['28', '35', '38', '42', '43', '52', '65', '76', '86', '150', '189', '192', '193', '195', '215', '269', '291', '320', '429', '465', '466', '482', '483', '510', '524', '541', '576', '582', '589', '603', '650', '680', '711', '722', '726', '783', '813', '820', '868', '869', '894', '1162', '1164', '1195', '1196', '1281']


### Implementando o BM25

In [8]:
from gensim.summarization.bm25 import BM25
from gensim.summarization.bm25 import get_bm25_weights

In [9]:
model_bm25 = BM25(doc_token)

In [10]:
#Calculando o IDF médio
idf_values = model_bm25.idf.values()
idf_sum = 0
for value in idf_values:
  idf_sum += value
idf_mean = idf_sum / len(idf_values)

In [11]:
#Calculando o score
scores = []
for query in queries_token:
  query_scores = model_bm25.get_scores(query,idf_mean)
  scores.append(query_scores)

In [ ]:
for i, query in enumerate(queries_token):
    ranked_documents = sorted(range(len(scores[i])), key=lambda j: scores[i][j], reverse=True)
    print("Consulta: ", query)
    print("Documentos mais relevantes: ", [k+1 for k in ranked_documents[:]])

### Calculando a acurácia

In [14]:
rel_set = {}
with open('/content/CISI.REL', 'r') as f:
    for l in f.readlines():
        qry_id = l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[0] 
        doc_id = l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[-1]

        if qry_id in rel_set:
            rel_set[qry_id].append(doc_id)
        else:
            rel_set[qry_id] = []
            rel_set[qry_id].append(doc_id)

In [52]:
model_return = []
model_return.append('')
ground_truth = []
ground_truth.append('')

for i, query in enumerate(queries_token):
  ranked_documents = sorted(range(len(scores[i])), key=lambda j: scores[i][j], reverse=True)
  if str(i+1) in rel_set:
    model_return.append([k+1 for k in ranked_documents[:len(rel_set[str(i+1)])]])
    ground_truth.append([int(k) for k in rel_set[str(i+1)]])
  else:
    model_return.append([])
    ground_truth.append([])

In [71]:
from sklearn.metrics import precision_score

precision_list = []
for i in range(1,len(queries)+1):
  acerto =0
  for k in model_return[i]:
    if k in ground_truth[i]:
      acerto += 1
  if ground_truth[i] != []:
    precision_list.append(acerto/len(ground_truth[i]))
  else:
    precision_list.append(-9)

precision_final =0
count = 0
for i in range(len(precision_list)):
  if precision_list[i] != -9:
    precision_final +=precision_list[i]
    count +=1

precision_final_2 = precision_final/count
print(precision_final_2)

0.2498303025107418
